In [ ]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
from datetime import datetime
from pytz import timezone

In [ ]:
def appendMod(var,column):
    if var is not None:
        column.append(var)
    else:
        column.append("NA")

In [ ]:
link = ("https://www.cricbuzz.com/cricket-series/6732/icc-cricket-world-cup-2023/matches")
page = req.get(link)
page.content
soup = bs(page.content, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <script>\n   var is_mobile = /symbian|tizen|midp|uc(web|browser)|MSIE (5.0|6.0|7.0|8.0)|tablet/i.test(navigator.userAgent);\tif(is_mobile && window.location.hostname != "www1.cricbuzz.com") window.location.hostname = "m.cricbuzz.com";\n  </script>\n  <style>\n   html{scroll-behavior: smooth;}\tbody{background:#E3E6E3; font-family: helvetica,"Segoe UI",Arial,sans-serif;color:#222;font-size:14px; line-height: 1.5; margin:0;}\tbody, .cb-comm-pg, .cb-hm-mid {min-height:1000px}\t.container{width:980px;margin:0 auto;}\t.page{max-width: 980px;margin: 0 auto;position: relative;}\t.cb-col-8 {width:8%;}\t.cb-col-10 {width:10%;}\t.cb-col-14 {width:14%;}\t.cb-col-16 {width:16%;}\t.cb-col-20 {width:20%;}\t.cb-col-25 {width:25%;}\t.cb-col-27 {width:27%;}\t.cb-col-30 {width:30%;}\t.cb-col-33 {width:33%;}\t.cb-col-40 {width:40%;}\t.cb-col-46 {width:46%;}\t.cb-col-47 {width:47%;}\

In [ ]:
dates=[]
timestamp=[]
teams=[]
team1=[]
team2=[]
location=[]
results=[]
GMTtimes=[]
localTimes=[]
round=[]
group=[]

prevTimestamp = ""

count = 0
total_matches = soup.findAll('div', attrs={'class':'cb-series-matches'})
for match1 in total_matches:
  count= count+1
  if count==46:
    appendMod("Semi final-1", round)
  elif count==47:
    appendMod("Semi final-2", round)
  elif count==48:
    appendMod("Final", round)
  else:
    appendMod("1", round)
  appendMod("Group A", group)
  # print(match1)
  dateSpan = match1.find('div', attrs={'class':'schedule-date'}).span
  tempDate = "0"
  if dateSpan is not None:
    tempDate = dateSpan['ng-bind'][1:dateSpan['ng-bind'].index('|')-3]
    appendMod(tempDate, timestamp)
    prevTimestamp = tempDate
  elif prevTimestamp:
    tempDate = prevTimestamp
    appendMod(prevTimestamp, timestamp)

  date = datetime.strftime(datetime.fromtimestamp(int(tempDate)),"%d-%m-%Y")
  appendMod(date, dates)



  # gmtTime = datetime.strftime(datetime.fromtimestamp(int(tempDate)),"%H:%M:%S")
  # appendMod(gmtTime, GMTtimes)

  # localTime = datetime.fromtimestamp(int(tempDate)).astimezone(timezone('Asia/Kolkata'))
  # print(localTime)

  data = match1.findAll('div', attrs={'class':'cb-srs-mtchs-tm'})
  vsTeam = data[0].find('a', attrs={'class':'text-hvr-underline'})
  if vsTeam is not None:
    temp = vsTeam.span.text
    temp = temp.split('vs')
    appendMod(vsTeam.span.text, teams)
    appendMod(temp[0].strip(),team1)
    appendMod(temp[1].strip().split(',')[0].strip(),team2)

  loc = data[0].find('div', attrs={'class':'text-gray'}).text
  appendMod(loc, location)

  result = data[0].find('a', attrs={'class':'cb-text-complete'})

  if result is not None:
    appendMod(result.text, results)
  else:
    appendMod("TBD", results)

  time = data[1].find('div', attrs={'class':'text-gray'}).text.split("/")
  # print(time[0].strip() + "," + time[1].strip())

  appendMod(time[0].strip()[0:8], GMTtimes)
  appendMod(time[1].strip()[0:8], localTimes)

appendMod("TBC vs TBC", teams)
appendMod("TBC",team1)
appendMod("TBC",team2)

In [ ]:
df=pd.DataFrame({'Timestamp':timestamp, 'Date': dates, 'Teams': teams, 'Location': location, 'Results': results, 'Time(GMT)': GMTtimes, 'Time(Local)': localTimes})

In [ ]:
df[['Team_1', 'Team_2']] = df['Teams'].str.split(' vs ', expand=True)
df['Team_2'] = df['Team_2'].str.replace(r', \d+.*', '')
df = df.drop(columns=['Teams'])

<ipython-input-6-3bb32429701e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Team_2'] = df['Team_2'].str.replace(r', \d+.*', '')


In [ ]:
df[['Winner', 'Margin']] = df['Results'].str.split(' won by ', expand=True)
df['Margin'] = df['Margin'].str.replace(r', \d+.*', '')
df = df.drop(columns=['Results'])

<ipython-input-7-822a98b7aa0e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Margin'] = df['Margin'].str.replace(r', \d+.*', '')


In [ ]:
df=df.drop(["Timestamp","Time(GMT)","Time(Local)","Winner","Margin"],axis=1)

In [ ]:
df_1=pd.DataFrame({'Round_number': round, 'Team_1': team1, 'Team_2': team2, 'Date': dates, 'Location': location, 'Group': group})

In [ ]:
df_1

,Round_number,Team_1,Team_2,Date,Location,Group
0,1,England,New Zealand,05-10-2023,"Narendra Modi Stadium, Ahmedabad",Group A
1,1,Pakistan,Netherlands,06-10-2023,"Rajiv Gandhi International Stadium, Hyderabad",Group A
2,1,Afghanistan,Bangladesh,07-10-2023,"Himachal Pradesh Cricket Association Stadium, ...",Group A
3,1,South Africa,Sri Lanka,07-10-2023,"Arun Jaitley Stadium, Delhi",Group A
4,1,Australia,India,08-10-2023,"MA Chidambaram Stadium, Chennai",Group A
5,1,New Zealand,Netherlands,09-10-2023,"Rajiv Gandhi International Stadium, Hyderabad",Group A
6,1,England,Bangladesh,10-10-2023,"Himachal Pradesh Cricket Association Stadium, ...",Group A
7,1,Sri Lanka,Pakistan,10-10-2023,"Rajiv Gandhi International Stadium, Hyderabad",Group A
8,1,Afghanistan,India,11-10-2023,"Arun Jaitley Stadium, Delhi",Group A
9,1,South Africa,Australia,12-10-2023,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Group A


In [ ]:
df.to_csv("Fixtures.csv")